## Q2: Egocentric

Write a github action that lets you commit only if README.md contains your name.

In [15]:
#Here I'm taking inspiration from the manual: https://docs.github.com/en/actions/writing-workflows/workflow-syntax-for-github-actions#jobs
def create_yaml_instruction_file():
    with open('../.github/workflows/check_readme_name.yml', 'w') as f:
        f.write('''
name: Check README at commit
run-name: ${{ github.actor }} running action is Massimo Girola in the README?
on: [push]
jobs:
  Check-README:
    runs-on: ubuntu-latest
    outputs:
      output1: ${{ steps.check-readme-step.outputs.output1 }}
    steps:
      - run: echo "🎉 The job was automatically triggered by a ${{ github.event_name }} event."
      - run: echo "🐧 This job is now running on a ${{ runner.os }} server hosted by GitHub!"
      - run: echo "🔎 The name of your branch is ${{ github.ref }} and your repository is ${{ github.repository }}."
      - name: Check out repository code
        uses: actions/checkout@v4
      - run: echo "💡 The ${{ github.repository }} repository has been cloned to the runner."
      - run: echo "🖥️ The workflow is now ready to test your code on the runner."
      - id: check-readme-step  
        run: |
          if grep "Massimo Girola" README.md; then
            echo "output1=Massimo Girola is in the README" >> "$GITHUB_OUTPUT"
          else
            echo "output1=Massimo Girola is not in the README" >> "$GITHUB_OUTPUT"
          fi
      - run: echo "🍏 This job's status is ${{ job.status }}." 
      
  Authorize-the-Commit:
    runs-on: ubuntu-latest
    needs: Check-README
    steps:
      - env: 
            OUTPUT1: ${{ needs.Check-README.outputs.output1 }}
        run: |
            echo "The output of the previous job was $OUTPUT1"
            if ! grep -q "Massimo Girola" README.md; then
                echo "Error: README.md does not contain 'Massimo Girola'."
                git reset --hard HEAD~1 #since I cannot run the action before the commit or the push, I need to reset the last commit if the condition is not met. The exercise asks to let commit only if the name is in the README but this is not possible using actions that can happen only after the push.
                git push --force
                exit 1
            else
                echo "'Massimo Girola' found in README.md. Proceeding with the commit."
            fi
    ''')

create_yaml_instruction_file()